In [1]:
import os
import re
import vrplib
import random
import matplotlib.pyplot as plt
import numpy as np
from collections import deque


In [2]:
#按文件名排序加载数据
def read_all_instances(root_folder, ending='.tsp'):  
    instances = []  
    
    def extract_k_number(file_name):  
        match = re.search(r'k(\d+)', file_name)  
        if match:  
            return int(match.group(1))
        return float('inf')  

    file_names = sorted(  
        [file_name for file_name in os.listdir(root_folder) if file_name.endswith(ending)],  
        key=extract_k_number 
    )   
    for file_name in file_names:  
        instance = vrplib.read_instance(str(os.path.join(root_folder, file_name))) 
        if instance:  
            instances.append(instance)  
            print(f'Successfully read {file_name}')  
        else:
            print(f'Failed to read {file_name}')  
    
    return instances  

In [10]:
root_folder = './data/cvrp/new_data/'  
cvrp_instances = read_all_instances(root_folder, ending='.vrp')

Successfully read A-n34-k5.vrp
Successfully read A-n32-k5.vrp
Successfully read A-n36-k5.vrp
Successfully read A-n33-k5.vrp
Successfully read A-n37-k5.vrp
Successfully read A-n38-k5.vrp
Successfully read A-n39-k5.vrp
Successfully read A-n39-k6.vrp
Successfully read A-n37-k6.vrp
Successfully read A-n33-k6.vrp
Successfully read A-n44-k6.vrp
Successfully read A-n45-k6.vrp
Successfully read A-n53-k7.vrp
Successfully read A-n46-k7.vrp
Successfully read A-n54-k7.vrp
Successfully read A-n45-k7.vrp
Successfully read A-n48-k7.vrp
Successfully read A-n62-k8.vrp
Successfully read A-n65-k9.vrp
Successfully read A-n69-k9.vrp
Successfully read A-n61-k9.vrp
Successfully read A-n64-k9.vrp
Successfully read A-n55-k9.vrp
Successfully read A-n60-k9.vrp
Successfully read A-n63-k9.vrp
Successfully read A-n63-k10.vrp
Successfully read A-n80-k10.vrp


In [3]:
# Function to visualize solution
def plot_CVRP_solution(routes, node_coords):
    plt.figure(figsize=(10, 8))
    
    # Plot nodes
    for i, (x, y) in enumerate(node_coords):
        plt.scatter(x, y, c='blue' if i == 0 else 'red')
        plt.text(x, y, f'{i}', fontsize=9, ha='right')
    
    # Plot routes
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for vehicle, route in enumerate(routes):
        route_coords = [node_coords[0]] + [node_coords[node] for node in route] + [node_coords[0]]
        x_coords, y_coords = zip(*route_coords)
        plt.plot(x_coords, y_coords, c=colors[vehicle % len(colors)], label=f'Vehicle {vehicle + 1}')
    
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Vehicle Routing Problem Solution')
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to test vehicle capacity constraint
def test_capacity_constraint(routes, demands, vehicle_capacity):
    for vehicle, route in enumerate(routes):
        total_demand = sum(demands[node] for node in route)
        if total_demand > vehicle_capacity:
            print(f"Vehicle {vehicle + 1} exceeds capacity: {total_demand} > {vehicle_capacity}")
        else:
            print(f"Vehicle {vehicle + 1} is within capacity: {total_demand} <= {vehicle_capacity}")


# Function to calculate route distance
def calculate_route_distance(route, distance_matrix):
    distance = 0
    if route:
        distance += distance_matrix[0][route[0]]  # From depot to first node
        for i in range(1, len(route)):
            distance += distance_matrix[route[i-1]][route[i]]
        distance += distance_matrix[route[-1]][0]  # From last node back to depot
    return distance

# Function to calculate total distance of all routes (CVRP)
def total_distance(routes, distance_matrix):
    return sum(calculate_route_distance(route, distance_matrix) for route in routes)


# Generate initial solution using a greedy approach
def greedy_initial_solution():
    routes = [[] for _ in range(num_vehicles)]
    current_load = [0] * num_vehicles
    visited = [False] * num_nodes
    visited[0] = True  # Starting from the depot

    for vehicle in range(num_vehicles):
        current_node = 0
        while True:
            next_node = None
            min_distance = float('inf')
            for i in range(1, num_nodes):
                if not visited[i] and current_load[vehicle] + demands[i] <= vehicle_capacity:
                    if distance_matrix[current_node][i] < min_distance:
                        min_distance = distance_matrix[current_node][i]
                        next_node = i
            if next_node is None:
                break
            routes[vehicle].append(next_node)
            current_load[vehicle] += demands[next_node]
            visited[next_node] = True
            current_node = next_node
    return routes

In [7]:

# a function for checking feasibility
def is_feasible(route, demands, vehicle_capacity):
    total_demand = sum(demands[node] for node in route)
    return total_demand <= vehicle_capacity # True or False

# new algorithm for solving CVRPs with local search 
def generate_neighbors(routes, demands, vehicle_capacity, num_rand_swaps = 10):
    neighbors = []
    for i in range(num_vehicles):
        for j in range(i +1, num_vehicles):
            # Route 1 = [1,2,3], Route 2 = [4,5, 6] -> Swap across routes [2,5]. New route 1: [1,5,3], new route 2: [4,2,6]
            for _ in range(num_rand_swaps):
                new_routes = [route.copy() for route in routes]
                if new_routes[i] and new_routes[j]:
                    node_i = random.choice(new_routes[i])
                    node_j = random.choice(new_routes[j])
                    idx_i = new_routes[i].index(node_i)
                    idx_j = new_routes[j].index(node_j)
                    new_routes[i][idx_i], new_routes[j][idx_j] = new_routes[j][idx_j], new_routes[i][idx_i]

                    # check for feasibility
                    if is_feasible(new_routes[i], demands, vehicle_capacity) and is_feasible(new_routes[j], demands, vehicle_capacity):
                        neighbors.append(new_routes)
    return neighbors


In [19]:
def simulated_annealing(initial_routes, demands, vehicle_capacity, initial_temp=5000, cooling_rate=0.995, stopping_temp=1):
    current_routes = initial_routes
    best_routes = initial_routes
    current_distance = total_distance(initial_routes, distance_matrix)
    best_distance = current_distance
    temperature = initial_temp

    while temperature > stopping_temp:
        neighbors = generate_neighbors(current_routes, demands, vehicle_capacity)
        # sort neighbors by total distance
        neighbors.sort(key=lambda x: total_distance(x, distance_matrix))
        neighbor = neighbors[0]
        neighbor_distance = total_distance(neighbor, distance_matrix)

        if neighbor_distance < current_distance or random.random() < np.exp((current_distance - neighbor_distance) / temperature):
            current_routes = neighbor
            current_distance = neighbor_distance
            
            if current_distance < best_distance:
                best_routes = current_routes
                best_distance = current_distance

        temperature *= cooling_rate

    return best_routes, best_distance

# Small

In [36]:
initial_temp = 5000
cooling_rate = 0.998
stopping_temp = 0.1

#root_folder = './data/cvrp/new_data/'  
#root_folder = './data/cvrp/solved'  
root_folder = './data/cvrp/small'  
#root_folder = './data/cvrp/medium'  
#root_folder = './data/cvrp/large'  
cvrp_instances = read_all_instances(root_folder, ending='.vrp')

for i in range(len(cvrp_instances)):
    cvrp_instance = cvrp_instances[i]
    vehicle_capacity = cvrp_instance['capacity']
    demands = cvrp_instance['demand']
    distance_matrix = [[int(value) for value in row] for row in cvrp_instance['edge_weight']]
    node_coords = cvrp_instance['node_coord']
    print("=====================",cvrp_instance['comment'],"=====================")
    match = re.search(r"No of trucks: (\d+)", str(cvrp_instance['comment']))  
    if match:  
        num_vehicles = int(match.group(1))  
    else:
        match_min_no_of_trucks = re.search(r"Min no of trucks: (\d+)", str(cvrp_instance['comment']))  
        if match_min_no_of_trucks:  
            num_vehicles = int(match_min_no_of_trucks.group(1))  
        else:  
            # 如果两种匹配都失败，返回 None 或处理异常  
            num_vehicles = None  

    num_nodes = len(node_coords)
    print("Number of Nodes:",num_nodes)
    initial_routes = greedy_initial_solution()
    optimized_routes_sa, optimized_distance_sa = simulated_annealing(initial_routes, demands, vehicle_capacity, initial_temp, cooling_rate, stopping_temp)
    
    print("Final Solution SA: ", optimized_routes_sa)
    print("Final Distance SA: ", optimized_distance_sa)
    
    # Plot and test the solution
    #plot_CVRP_solution(optimized_routes_sa, node_coords)
    test_capacity_constraint(optimized_routes_sa, demands, vehicle_capacity)

Successfully read A-n32-k5.vrp
Successfully read A-n37-k5.vrp
Successfully read A-n48-k7.vrp
Successfully read P-n50-k10.vrp
===================== (Augerat et al, No of trucks: 5, Optimal value: 784) =====================
Number of Nodes: 32
Final Solution SA:  [[30, 26, 12, 1, 7, 29, 22, 11, 8, 18], [14, 24, 15, 10, 25, 5, 20], [6, 3, 2, 23, 28, 4, 9], [17, 21, 31, 19, 13, 16], [27]]
Final Distance SA:  960
Vehicle 1 is within capacity: 99 <= 100
Vehicle 2 is within capacity: 96 <= 100
Vehicle 3 is within capacity: 97 <= 100
Vehicle 4 is within capacity: 98 <= 100
Vehicle 5 is within capacity: 20 <= 100
===================== (Augerat et al, No of trucks: 5, Optimal value: 669) =====================
Number of Nodes: 37
Final Solution SA:  [[15, 16, 22, 7, 4, 33, 5, 6, 10], [34, 29, 32, 28, 31, 26, 30, 35, 18, 25], [21, 1, 14, 23, 20, 19, 2, 17], [13, 12, 11, 8, 27, 9, 24, 3], [36]]
Final Distance SA:  762
Vehicle 1 is within capacity: 98 <= 100
Vehicle 2 is within capacity: 100 <= 100


# Medium

In [21]:
initial_temp = 5000
cooling_rate = 0.998
stopping_temp = 0.1

#root_folder = './data/cvrp/new_data/'  
#root_folder = './data/cvrp/solved'  
#root_folder = './data/cvrp/small'  
root_folder = './data/cvrp/medium'  
#root_folder = './data/cvrp/large'  
cvrp_instances = read_all_instances(root_folder, ending='.vrp')

for i in range(len(cvrp_instances)):
    cvrp_instance = cvrp_instances[i]
    vehicle_capacity = cvrp_instance['capacity']
    demands = cvrp_instance['demand']
    distance_matrix = [[int(value) for value in row] for row in cvrp_instance['edge_weight']]
    node_coords = cvrp_instance['node_coord']
    print("=====================",cvrp_instance['comment'],"=====================")
    match = re.search(r"No of trucks: (\d+)", str(cvrp_instance['comment']))  
    if match:  
        num_vehicles = int(match.group(1))  
    else:
        match_min_no_of_trucks = re.search(r"Min no of trucks: (\d+)", str(cvrp_instance['comment']))  
        if match_min_no_of_trucks:  
            num_vehicles = int(match_min_no_of_trucks.group(1))  
        else:  
            # 如果两种匹配都失败，返回 None 或处理异常  
            num_vehicles = None  

    num_nodes = len(node_coords)
    print("Number of Nodes:",num_nodes)
    initial_routes = greedy_initial_solution()
    optimized_routes_sa, optimized_distance_sa = simulated_annealing(initial_routes, demands, vehicle_capacity, initial_temp, cooling_rate, stopping_temp)
    
    print("Final Solution SA: ", optimized_routes_sa)
    print("Final Distance SA: ", optimized_distance_sa)
    
    # Plot and test the solution
    #plot_CVRP_solution(optimized_routes_sa, node_coords)
    test_capacity_constraint(optimized_routes_sa, demands, vehicle_capacity)

Successfully read A-n53-k7.vrp
Successfully read A-n63-k10.vrp
Successfully read P-n65-k10.vrp
Successfully read A-n80-k10.vrp
Successfully read E-n76-k14.vrp
===================== (Augerat et al, No of trucks: 7, Optimal value: 1010) =====================
Number of Nodes: 53
Final Solution SA:  [[1, 4, 22, 30, 44, 40, 35, 38, 18, 27, 51], [31, 20, 6, 10, 26, 49, 29, 28], [47, 41, 24, 11, 52, 13], [3, 5, 14, 34, 21, 25, 39], [7, 17, 9, 12, 16, 32, 48, 15, 19, 50, 36], [37, 45, 43, 23, 42, 2], [46, 8, 33]]
Final Distance SA:  1191
Vehicle 1 is within capacity: 100 <= 100
Vehicle 2 is within capacity: 100 <= 100
Vehicle 3 is within capacity: 94 <= 100
Vehicle 4 is within capacity: 91 <= 100
Vehicle 5 is within capacity: 100 <= 100
Vehicle 6 is within capacity: 98 <= 100
Vehicle 7 is within capacity: 81 <= 100
===================== (Augerat et al, No of trucks: 10, Optimal value: 1314) =====================
Number of Nodes: 63
Final Solution SA:  [[52, 30, 4, 38, 26, 50, 62], [31, 47, 34,

# Large

In [37]:
initial_temp = 5000
cooling_rate = 0.998
stopping_temp = 0.1

#root_folder = './data/cvrp/new_data/'  
#root_folder = './data/cvrp/solved'  
#root_folder = './data/cvrp/small'  
#root_folder = './data/cvrp/medium'  
root_folder = './data/cvrp/large'  
cvrp_instances = read_all_instances(root_folder, ending='.vrp')

for i in range(len(cvrp_instances)):
    cvrp_instance = cvrp_instances[i]
    vehicle_capacity = cvrp_instance['capacity']
    demands = cvrp_instance['demand']
    distance_matrix = [[int(value) for value in row] for row in cvrp_instance['edge_weight']]
    node_coords = cvrp_instance['node_coord']
    print("=====================",cvrp_instance['comment'],"=====================")
    match = re.search(r"No of trucks: (\d+)", str(cvrp_instance['comment']))  
    if match:  
        num_vehicles = int(match.group(1))  
    else:
        match_min_no_of_trucks = re.search(r"Min no of trucks: (\d+)", str(cvrp_instance['comment']))  
        if match_min_no_of_trucks:  
            num_vehicles = int(match_min_no_of_trucks.group(1))  
        else:  
            # 如果两种匹配都失败，返回 None 或处理异常  
            num_vehicles = None  

    num_nodes = len(node_coords)
    print("Number of Nodes:",num_nodes)
    initial_routes = greedy_initial_solution()
    optimized_routes_sa, optimized_distance_sa = simulated_annealing(initial_routes, demands, vehicle_capacity, initial_temp, cooling_rate, stopping_temp)
    
    print("Final Solution SA: ", optimized_routes_sa)
    print("Final Distance SA: ", optimized_distance_sa)
    
    # Plot and test the solution
    #plot_CVRP_solution(optimized_routes_sa, node_coords)
    test_capacity_constraint(optimized_routes_sa, demands, vehicle_capacity)

Successfully read X-n120-k6.vrp
Successfully read X-n115-k10.vrp
Successfully read X-n139-k10.vrp
Successfully read X-n110-k13.vrp
===================== "Generated by Uchoa, Pecin, Pessoa, Poggi, Subramanian, and Vidal (2013),, Min no of trucks: 6" =====================
Number of Nodes: 120
Final Solution SA:  [[20, 107, 96, 71, 10, 88, 94, 61, 119, 84, 16, 32, 91, 21, 118, 30, 67, 73, 31, 52, 50], [55, 70, 3, 45, 8, 90, 114, 78, 109, 76, 43, 113, 2, 49, 39, 82, 64, 98, 13, 42, 93], [38, 29, 25, 33, 23, 14, 36, 115, 48, 86, 110, 74, 65, 106, 7, 97, 19, 28, 83, 58, 79], [54, 87, 72, 102, 111, 34, 11, 27, 56, 1, 24, 37, 59, 60, 68, 81, 69, 100, 77, 57, 53], [116, 112, 62, 40, 6, 92, 35, 22, 4, 95, 89, 85, 51, 63, 104, 105, 101, 47, 26, 5, 41], [80, 12, 15, 44, 75, 46, 66, 108, 117, 103, 17, 18, 9, 99]]
Final Distance SA:  15398
Vehicle 1 is within capacity: 21 <= 21
Vehicle 2 is within capacity: 21 <= 21
Vehicle 3 is within capacity: 21 <= 21
Vehicle 4 is within capacity: 21 <= 21
Vehicl

# Solved

In [18]:
initial_temp = 5000
cooling_rate = 0.998
stopping_temp = 0.1

#root_folder = './data/cvrp/new_data/'  
root_folder = './data/cvrp/solved'  
#root_folder = './data/cvrp/small'  
#root_folder = './data/cvrp/medium'  
#root_folder = './data/cvrp/large'  
cvrp_instances = read_all_instances(root_folder, ending='.vrp')

for i in range(len(cvrp_instances)):
    cvrp_instance = cvrp_instances[i]
    vehicle_capacity = cvrp_instance['capacity']
    demands = cvrp_instance['demand']
    distance_matrix = [[int(value) for value in row] for row in cvrp_instance['edge_weight']]
    node_coords = cvrp_instance['node_coord']
    print("=====================",cvrp_instance['comment'],"=====================")
    match = re.search(r"No of trucks: (\d+)", str(cvrp_instance['comment']))  
    if match:  
        num_vehicles = int(match.group(1))  
    else:
        match_min_no_of_trucks = re.search(r"Min no of trucks: (\d+)", str(cvrp_instance['comment']))  
        if match_min_no_of_trucks:  
            num_vehicles = int(match_min_no_of_trucks.group(1))  
        else:  
            # 如果两种匹配都失败，返回 None 或处理异常  
            num_vehicles = None  

    num_nodes = len(node_coords)
    print("Number of Nodes:",num_nodes)
    initial_routes = greedy_initial_solution()
    optimized_routes_sa, optimized_distance_sa = simulated_annealing(initial_routes, demands, vehicle_capacity, initial_temp, cooling_rate, stopping_temp)
    
    print("Final Solution SA: ", optimized_routes_sa)
    print("Final Distance SA: ", optimized_distance_sa)
    
    # Plot and test the solution
    #plot_CVRP_solution(optimized_routes_sa, node_coords)
    test_capacity_constraint(optimized_routes_sa, demands, vehicle_capacity)

Successfully read A-n45-k6.vrp
Successfully read P-n60-k10.vrp
Successfully read E-n101-k14.vrp
===================== (Augerat et al, No of trucks: 6, Optimal value: 944) =====================
Number of Nodes: 45
Final Solution SA:  [[20, 3, 10, 41, 33, 21, 8, 5], [32, 7, 18, 17, 40, 34, 11, 19, 26], [23, 9, 28, 13, 22, 25, 15, 2], [14, 31, 35, 1, 6, 44], [29, 43, 27, 37, 24], [38, 12, 39, 36, 42, 16, 4]]
Final Distance SA:  1047
Vehicle 1 is within capacity: 90 <= 100
Vehicle 2 is within capacity: 99 <= 100
Vehicle 3 is within capacity: 100 <= 100
Vehicle 4 is within capacity: 100 <= 100
Vehicle 5 is within capacity: 98 <= 100
Vehicle 6 is within capacity: 95 <= 100
===================== (Augerat et al, No of trucks: 10, Optimal value: 744) =====================
Number of Nodes: 60
Final Solution SA:  [[50, 18, 24, 44, 3, 51], [27, 52, 34, 46, 8, 26], [45, 57, 20, 37, 5, 48], [15, 13, 54, 19, 14, 53, 7], [16, 56, 43, 41, 42, 1, 23, 49], [32, 39, 9, 31, 55, 25], [58, 10, 38, 11, 35], [